In [19]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

from sklearn.model_selection import train_test_split,GridSearchCV
from sklearn.metrics import accuracy_score
from sklearn.compose import ColumnTransformer
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler,MinMaxScaler,OneHotEncoder
from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import Pipeline



In [20]:
df = pd.read_csv('train _titanic.csv')

In [21]:
df.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [22]:
df.drop(columns=['PassengerId','Name','Ticket','Cabin'],inplace=True)

In [23]:
df.head()

,Survived,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked
0,0,3,male,22.0,1,0,7.2500,S
1,1,1,female,38.0,1,0,71.2833,C
2,1,3,female,26.0,0,0,7.9250,S
3,1,1,female,35.0,1,0,53.1000,S
4,0,3,male,35.0,0,0,8.0500,S


In [24]:
X = df.drop(columns=['Survived'])
Y = df['Survived']

In [25]:
X_train,X_test,Y_train,Y_test = train_test_split(X,Y,test_size=0.2,random_state=2)

In [26]:
X_train.head()

,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked
30,1,male,40.0,0,0,27.7208,C
10,3,female,4.0,1,1,16.7000,S
873,3,male,47.0,0,0,9.0000,S
182,3,male,9.0,4,2,31.3875,S
876,3,male,20.0,0,0,9.8458,S


In [27]:
numerical_feature = ['Age','Fare']
numerical_transformer = Pipeline(steps=
    [
        ('imputer',SimpleImputer(strategy='median')),
        ('scaler',StandardScaler())
    ]
)

categorical_features = ['Embarked','Sex']
categorical_transformer = Pipeline(
    steps = [
        ('imputer',SimpleImputer(strategy=',ost_frequent')),
        ('ohe',OneHotEncoder(handle_unknown='ignore'))
    ]
)

In [28]:
prepocessor = ColumnTransformer(
    transformers=[
        ('num',numerical_transformer,numerical_feature),
        ('cat', categorical_transformer,categorical_features)
    ]
)

In [29]:
clf = Pipeline(
    steps = [
        ('prepocessor',prepocessor),
        ('classifier',LogisticRegression())
    ]
)

In [30]:
clf

Pipeline(steps=[('prepocessor',
                 ColumnTransformer(transformers=[('num',
                                                  Pipeline(steps=[('imputer',
                                                                   SimpleImputer(strategy='median')),
                                                                  ('scaler',
                                                                   StandardScaler())]),
                                                  ['Age', 'Fare']),
                                                 ('cat',
                                                  Pipeline(steps=[('imputer',
                                                                   SimpleImputer(strategy=',ost_frequent')),
                                                                  ('ohe',
                                                                   OneHotEncoder(handle_unknown='ignore'))]),
                                                  ['Embarked', 'Sex'])])),
                ('classifier', LogisticRegression())])

In [31]:
from sklearn import set_config

set_config(display='diagram')
clf

Pipeline(steps=[('prepocessor',
                 ColumnTransformer(transformers=[('num',
                                                  Pipeline(steps=[('imputer',
                                                                   SimpleImputer(strategy='median')),
                                                                  ('scaler',
                                                                   StandardScaler())]),
                                                  ['Age', 'Fare']),
                                                 ('cat',
                                                  Pipeline(steps=[('imputer',
                                                                   SimpleImputer(strategy=',ost_frequent')),
                                                                  ('ohe',
                                                                   OneHotEncoder(handle_unknown='ignore'))]),
                                                  ['Embarked', 'Sex'])])),
                ('classifier', LogisticRegression())])

## Grid Serach CV (for finding best value)

In [32]:
param_grid = {
    'prepocessor__num__imputer__strategy':['mean','median'],
    'prepocessor__cat__imputer__strategy':['most_frequent','constant'],
    'classifier__C':[0.1,0.2,10,20]
}

grid_search = GridSearchCV(clf,param_grid,cv=10)

In [34]:
grid_search.fit(X_train,Y_train)

print(f'Best params:')
print(grid_search.best_params_)

Best params:
{'classifier__C': 0.2, 'prepocessor__cat__imputer__strategy': 'most_frequent', 'prepocessor__num__imputer__strategy': 'mean'}


In [35]:
print(f'Internal CV Score:{grid_search.best_score_:.3f}')

Internal CV Score:0.788


In [37]:
cv_result = pd.DataFrame(grid_search.cv_results_)
cv_result = cv_result.sort_values("mean_test_score",ascending=False)
cv_result

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_classifier__C,param_prepocessor__cat__imputer__strategy,param_prepocessor__num__imputer__strategy,params,split0_test_score,split1_test_score,...,split3_test_score,split4_test_score,split5_test_score,split6_test_score,split7_test_score,split8_test_score,split9_test_score,mean_test_score,std_test_score,rank_test_score
4,0.022701,0.004695,0.008293,0.001962,0.2,most_frequent,mean,"{'classifier__C': 0.2, 'prepocessor__cat__impu...",0.763889,0.861111,...,0.802817,0.788732,0.760563,0.84507,0.732394,0.760563,0.788732,0.787852,0.037656,1
5,0.015814,0.000648,0.006827,0.001350,0.2,most_frequent,median,"{'classifier__C': 0.2, 'prepocessor__cat__impu...",0.763889,0.861111,...,0.802817,0.788732,0.760563,0.84507,0.732394,0.760563,0.788732,0.787852,0.037656,1
6,0.019337,0.004903,0.007836,0.003177,0.2,constant,mean,"{'classifier__C': 0.2, 'prepocessor__cat__impu...",0.763889,0.861111,...,0.802817,0.788732,0.760563,0.84507,0.732394,0.760563,0.788732,0.787852,0.037656,1
7,0.017843,0.001140,0.007048,0.001949,0.2,constant,median,"{'classifier__C': 0.2, 'prepocessor__cat__impu...",0.763889,0.861111,...,0.802817,0.788732,0.760563,0.84507,0.732394,0.760563,0.788732,0.787852,0.037656,1
8,0.017278,0.001130,0.007007,0.001246,10,most_frequent,mean,"{'classifier__C': 10, 'prepocessor__cat__imput...",0.763889,0.861111,...,0.802817,0.788732,0.760563,0.84507,0.732394,0.760563,0.788732,0.787852,0.037656,1
9,0.018368,0.001331,0.006611,0.001764,10,most_frequent,median,"{'classifier__C': 10, 'prepocessor__cat__imput...",0.763889,0.861111,...,0.802817,0.788732,0.760563,0.84507,0.732394,0.760563,0.788732,0.787852,0.037656,1
10,0.017775,0.001173,0.006873,0.001863,10,constant,mean,"{'classifier__C': 10, 'prepocessor__cat__imput...",0.763889,0.861111,...,0.802817,0.788732,0.760563,0.84507,0.732394,0.760563,0.788732,0.787852,0.037656,1
11,0.018123,0.001634,0.006442,0.001046,10,constant,median,"{'classifier__C': 10, 'prepocessor__cat__imput...",0.763889,0.861111,...,0.802817,0.788732,0.760563,0.84507,0.732394,0.760563,0.788732,0.787852,0.037656,1
12,0.016741,0.001873,0.006502,0.001599,20,most_frequent,mean,"{'classifier__C': 20, 'prepocessor__cat__imput...",0.763889,0.861111,...,0.802817,0.788732,0.760563,0.84507,0.732394,0.760563,0.788732,0.787852,0.037656,1
13,0.016460,0.001005,0.006737,0.001995,20,most_frequent,median,"{'classifier__C': 20, 'prepocessor__cat__imput...",0.763889,0.861111,...,0.802817,0.788732,0.760563,0.84507,0.732394,0.760563,0.788732,0.787852,0.037656,1
